#  CONVOLUTIONAL NEURAL NETWORK

### Loads the dependencies


In [1]:
import os          
import cv2                                 
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf
from sklearn.metrics import *         
import matplotlib.pyplot as plt 
import sklearn.model_selection as model_selection

2022-12-13 08:27:48.074711: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


### Load the raw data

In [2]:
def load_data(dataset):
    class_names = []
    images = []
    labels = []  
    for folder in os.listdir(dataset):
        class_names.append(folder)    
    class_names_label = {class_name:i for i, class_name in enumerate(class_names)} 
    print("Loading Dataset {}".format(dataset))
    for folder in os.listdir(dataset):
        label = class_names_label[folder]
        for file in tqdm(os.listdir(os.path.join(dataset, folder))):
            img_path = os.path.join(os.path.join(dataset, folder), file)
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            IMAGE_SIZE = (150, 150)
            image = cv2.resize(image, IMAGE_SIZE)
            images.append(image)
            labels.append(label)
    return images, labels , class_names

dataset = '../datasets/'
images, labels, class_names = load_data(dataset)

Loading Dataset ../datasets/


100%|███████████████████████████████████████████| 56/56 [00:01<00:00, 53.96it/s]


### Normalizing

In [3]:
images = np.array(images, dtype = 'float32')
labels = np.array(labels, dtype = 'int32') 
images = images/255.0

### Train Test Split

In [4]:
train_images, test_images, train_labels, test_labels = model_selection.train_test_split(images, labels, train_size=0.80,test_size=0.20, random_state=101) 

###  Build the model_RR 

In [5]:
model_RR = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(30, activation=tf.nn.softmax)
])

2022-12-13 08:28:29.841048: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-12-13 08:28:29.844894: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-12-13 08:28:29.905413: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-13 08:28:29.905738: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.515GHz coreCount: 14 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 178.84GiB/s
2022-12-13 08:28:29.905785: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-12-13 08:28:29.925573: I tensorflow/stream_executor/platform/de

###  Compile the model_RR 

In [6]:
model_RR.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

###  Train the  model_RR

In [7]:
history_RR = model_RR.fit(train_images, train_labels, batch_size=128, epochs=5, validation_split=0.2)

2022-12-13 08:28:33.123954: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 316980000 exceeds 10% of free system memory.
2022-12-13 08:28:33.308545: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-12-13 08:28:33.310984: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2994300000 Hz


Epoch 1/5


2022-12-13 08:28:33.603752: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-12-13 08:28:33.935082: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-12-13 08:28:34.707668: E tensorflow/stream_executor/cuda/cuda_dnn.cc:336] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2022-12-13 08:28:34.736167: E tensorflow/stream_executor/cuda/cuda_dnn.cc:336] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2022-12-13 08:28:34.736247: W tensorflow/core/framework/op_kernel.cc:1763] OP_REQUIRES failed at conv_ops_fused_impl.h:697 : Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Relu (defined at tmp/ipykernel_3959/514245135.py:1) ]] [Op:__inference_train_function_726]

Function call stack:
train_function


### Evaluate the  model_RR

In [8]:
model_RR_results = model.evaluate(test_images, test_labels)

NameError: name 'model' is not defined

### Predictions model_RR

In [9]:
predictions = np.argmax(model_RR.predict(test_images), axis = 1)
actual = test_labels

2022-12-13 08:28:39.148886: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 99090000 exceeds 10% of free system memory.
2022-12-13 08:28:39.353745: E tensorflow/stream_executor/cuda/cuda_dnn.cc:336] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2022-12-13 08:28:39.358443: E tensorflow/stream_executor/cuda/cuda_dnn.cc:336] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2022-12-13 08:28:39.358505: W tensorflow/core/framework/op_kernel.cc:1763] OP_REQUIRES failed at conv_ops_fused_impl.h:697 : Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Relu (defined at tmp/ipykernel_3959/2605699045.py:1) ]] [Op:__inference_predict_function_822]

Function call stack:
predict_function


### Confusion matrix model_RR

In [ ]:
confusion_matrix_df = pd.DataFrame(confusion_matrix(actual,predictions),index = class_names, columns =class_names)

plt.figure(figsize=(15,15))
sns.heatmap(confusion_matrix_df, annot=True)
plt.title('Confusion Matrix')
plt.ylabel('Actal Values')
plt.xlabel('Predicted Values')
plt.show()

##  Comparison

###  Create a dataframe

In [ ]:
qnn_history.history
accuracy = pd.DataFrame()

accuracy['QNN_accuracy']  = qnn_history.history['hinge_accuracy']
accuracy['QNN_val_accuracy'] = qnn_history.history['val_hinge_accuracy']

accuracy['CNN_accuracy']  = cnn_history.history['accuracy']
accuracy['CNN_val_accuracy'] = cnn_history.history['val_accuracy']

accuracy['CNN_accuracy']  = cnn_history.history['accuracy']
accuracy['CNN_val_accuracy'] = cnn_history.history['val_accuracy']

accuracy['CNN_accuracy']  = cnn_history.history['accuracy']
accuracy['CNN_val_accuracy'] = cnn_history.history['val_accuracy']

accuracy['CNN_accuracy']  = cnn_history.history['accuracy']
accuracy['CNN_val_accuracy'] = cnn_history.history['val_accuracy']

accuracy['CNN_accuracy']  = cnn_history.history['accuracy']
accuracy['CNN_val_accuracy'] = cnn_history.history['val_accuracy']

accuracy['CNN_accuracy']  = cnn_history.history['accuracy']
accuracy['CNN_val_accuracy'] = cnn_history.history['val_accuracy']

accuracy['CNN_accuracy']  = cnn_history.history['accuracy']
accuracy['CNN_val_accuracy'] = cnn_history.history['val_accuracy']

accuracy['CNN_accuracy']  = cnn_history.history['accuracy']
accuracy['CNN_val_accuracy'] = cnn_history.history['val_accuracy']

accuracy['CNN_accuracy']  = cnn_history.history['accuracy']
accuracy['CNN_val_accuracy'] = cnn_history.history['val_accuracy']

accuracy['CNN_accuracy']  = cnn_history.history['accuracy']
accuracy['CNN_val_accuracy'] = cnn_history.history['val_accuracy']


print("Quantum CNN accuracy and  Classical CNN accuracy\n")
print(accuracy)
print("\nQuantum CNN loss and  Classical CNN loss\n")
print(loss)

### Comparison using line chart

In [ ]:
def plot_accuracy():
    plt.plot(accuracy['QNN_accuracy'],'b--',label = 'QNN_accuracy')
    plt.plot(accuracy['QNN_val_accuracy'],'r--',label = 'QNN_val_accuracy')
    plt.plot(accuracy['CNN_accuracy'],'g--',label = 'CNN_accuracy')
    plt.plot(accuracy['CNN_val_accuracy'],'y--',label = 'CNN_val_accuracy')
    plt.legend()
    plt.title('Quantum CNN accuracy Vs Classical CNN accuracy')
    plt.show()

In [ ]:
plot_accuracy()

### Comparison using Bar chart

In [ ]:
accuracy =[qnn_results[1],cnn_results[1]]
lable = ["Quantum", "Classical"]
barlist = plt.bar(lable,accuracy)
barlist[0].set_color('b')
barlist[1].set_color('r')
plt.title("Quantum Vs Classical")
plt.show()